In [1]:
import json
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
dong1 = ['광천동', '유덕동', '양동', '양3동', '농성1동', '농성2동']
dong2 = ['치평동', '상무1동','상무2동', '화정1동','화정2동','화정3동']
dong3 = ['화정4동','서창동','금호1동','금호2동','풍암동','동천동']
bigcode = ["G2", "I2", "S2", "R1", "P1", "L1", "I1", "M1", "Q1", "N1"]

In [ ]:
category_df = pd.read_csv("업종.csv", encoding='utf-8-sig')

In [2]:
columns = ['구', '업종코드', '업종이름', '검색업종', '23.12', '24.01', '24.02', '24.03', '24.04', 
           '24.05', '24.06', '24.07', '24.08', '24.09', '24.10', '24.11', '24.12']
delivery_sales_df  = pd.DataFrame(columns=columns)
store_sales_df  = pd.DataFrame(columns=columns)

In [3]:
store_sales_df.to_csv(f"../data/test_상가_매출.csv", encoding='utf-8-sig', index=False)

In [ ]:
s = Service(ChromeDriverManager().install())
driver = wb.Chrome(service=s)
driver.maximize_window()
driver.get('https://bigdata.sbiz.or.kr/gis/?type=detail&rptpType=gisDetail&lat=35.1287735&lng=126.8532396')

In [ ]:
#################################### 반복할 동 선택
seogu = dong1
####################################

In [ ]:
error_cate = []

for dong in seogu: #### 구 반복 ####
    while 1:
        time.sleep(1)
        search_box = driver.find_element(By.ID, "searchAddress")  # 검색창

        search_box.send_keys(Keys.CONTROL + "a")  # 동 입력창 초기화
        time.sleep(0.5)
        search_box.send_keys(Keys.DELETE)
        time.sleep(1)

        search_box.send_keys(f"광주 서구 {dong}")  # 입력창에 원하는 동 입력
        search_box.send_keys(Keys.RETURN)
        time.sleep(1)

        button = driver.find_element(By.CSS_SELECTOR, "ul#admListAddress>li>button") # 검색
        
        if button.text == f"광주 서구 {dong}":
            button.click()
            break                               # while 나가기 -> 동 검색 끝
        else:
            print("동 찾기 에러")
            time.sleep(1)

    for i in range(len(category_df)):           #### 세부업종 구간 ####
        time.sleep(1)
        Upjong = driver.find_element(By.ID, "selectedUpjong") # 업종 입력창

        Upjong.send_keys(Keys.CONTROL + "a")  # 업종 입력창 초기화
        Upjong.send_keys(Keys.DELETE)
        time.sleep(1)

        up_name = category_df["업종이름"].iloc[i] # 입력할 업종 불러오기
        up_code = category_df["업종코드"].iloc[i]

        Upjong.send_keys("> " + up_name)  # 입력창에 업종 검색
        Upjong.send_keys(Keys.RETURN)
        time.sleep(1)

        search = driver.find_elements(By.CSS_SELECTOR, "ul#tpbizSearchListUl>li>button")# 연관 검색 클릭
        for j in range(len(search)):
            if search[j].text.find(up_name) != -1:
                button = search[j]
                break
        button.click()
        time.sleep(1)

        button = driver.find_element(By.ID, "analysisBtn")  # 분석하기 버튼 클릭
        button.click()
        time.sleep(8)

        td_name= driver.find_elements(By.CSS_SELECTOR,'div.boxArea>div.summary>dl') # 상세 페이지에 있는 업종 명 가져오기
        for k in range(len(td_name)):                                               #    (제대로 가져왔는지 확인을 위해)
            if td_name[k].text.find(up_name) != -1:
                td_name = td_name[k].text

        # 상가 매출 데이터 가져오기
        tr = driver.find_elements(By.CSS_SELECTOR, "div.boxArea.type4.space>div.tableArea>div.scrollX>table>tbody>tr") # 상세 보고서 테이블들
        try:
            for td in tr: # 매출 테이블 찾기
                if td.text.find('분석영역 매출') != -1:
                    store_sales_data = td.text.split(" ")[2:] # 상가 매출 데이터 추가작업
                    store_sales_df.loc[i] = [dong, up_code, up_name, td_name] + store_sales_data # 최종 데이터 DF에 추가
                    break
        except:
            error_cate.append(td_name) # 매출 테이블을 찾지 못했을 경울

        # 배달 매출 데이터 가져오기
        try :                                                            # 배달 매출 txt 바로 리스트로 바꿔 넣기
            delivery_sales_data = driver.find_element(By.CSS_SELECTOR, "tr#genderAllSum").text.split(" ")[2:]
            delivery_sales_df.loc[i] = [dong, up_code, up_name, td_name] + delivery_sales_data # 최종 데이터 DF에 추가
        except:
            pass # 배달 매출 정보가 없을 경우 배달을 하지 않는 업종이므로 pass

        # 상세 분석 나가기
        try:
            close = driver.find_element(By.CSS_SELECTOR, "section#reportDiv>div.reportInner>button.close")
            close.click()
        except:
            time.sleep(3)
            close = driver.find_element(By.CSS_SELECTOR, "section#reportDiv>div.reportInner>button.close")
            close.click()

        print(f"{dong}, 진행 횟수: {i + 1}, 남은 횟수: {len(category_df) - (i + 1)}")

    ## 저장 후 DF초기화
    store_sales_df.to_csv(f"../data/01_상가/{dong}_상가_매출.csv", encoding='utf-8-sig', index=False)
    delivery_sales_df.to_csv(f"../data/02_배달/{dong}_배달_매출.csv", encoding='utf-8-sig', index=False)
    store_sales_df = pd.DataFrame(columns=columns)
    delivery_sales_df = pd.DataFrame(columns=columns)
    


In [ ]:
close = driver.find_element(By.CSS_SELECTOR, "section#reportDiv>div.reportInner>button.close")
close.click()

API를 이용해서 상가 분류를 중->소분류로 구분

In [ ]:
# Key = ''
# small_name = []
# small_code = []

# for i in bigcode:
#     middleUpjongList = f'http://apis.data.go.kr/B553077/api/open/sdsc2/middleUpjongList?serviceKey={Key}&indsLclsCd={i}&type=json'
#     middle_json = requests.get(middleUpjongList)
#     json_ob_1 = json.loads(middle_json.text)
#     middle_body = json_ob_1['body']['items']

#     for j in middle_body:
#         smallUpjongList = f'http://apis.data.go.kr/B553077/api/open/sdsc2/smallUpjongList?serviceKey={Key}&indsLclsCd={i}&indsMclsCd={j.get('indsMclsCd')}&type=json'
#         json_ob_2 = json.loads(requests.get(smallUpjongList).text)
#         small_body = json_ob_2['body']['items']
#         for k in small_body:
#             small_name.append(k.get('indsSclsNm'))
#             small_code.append(k.get('indsSclsCd'))
            
# print("업종코드 갯수 : {}".format(len(small_code)))

df를 불러와서 중복이 있는지 확인

In [ ]:
# df = pd.read_csv("농성1동_상가_매출.csv", encoding='utf-8-sig')
# err = []
# for i in range(len(df)):
#     if df.loc[i, '업종이름'] != df.loc[i, '검색업종'].split('> ')[-1]:
#         print(df.loc[i, '업종이름'])
#         err.append(df.loc[i, '업종이름'])

트러블 슈팅

소분류 업종을 입력했을 때 중복된 이름을 가진 다른 업종을 선택하게 되는 경우가 있었다 -> "업종이름" 에서 "> 업종이름" 으로 변경해서 수정